In [1]:
import requests, json, numpy, datetime
import pywren
import boto3
import numpy as np

In [2]:
GEOJSONS = ['train-borde_rural.geojson', 'train-borde_soacha.geojson',
            'train-castries.geojson', 'train-dennery.geojson',
            'train-gros_islet.geojson', 'train-mixco_1_and_ebenezer.geojson',
            'train-mixco_3.geojson']

In [3]:
BUCKET = 'mapping-disaster-risk'

TIFS = ['borde_rural_ortho-cog.tif', 'borde_soacha_ortho-cog.tif',
        'castries_ortho-cog.tif', 'dennery_ortho-cog.tif',
        'gros_islet_ortho-cog.tif', 'mixco_1_and_ebenezer_ortho-cog.tif',
        'mixco_3_ortho-cog.tif']

# Note that all geojsonn names are the object names as well, so to access an object call:
# https://mapping-disaster-risk.s3.amazonaws.com/train-borde_rural.geojson

In [17]:
# Step 1: Get all geojsons and create new footprint dictionaries
def get_geojsons(geojson):
    '''
    Load geojson, extract relevant information, return as a dictionary.
    '''
    try:
        s3 = boto3.client('s3', 'us-east-1')
        obj = s3.get_object(Bucket='mapping-disaster-risk', Key=geojson)
        geo_json = json.load(obj['Body'])
        polygons = []
        for feature in geo_json['features']:
            polygon = {}
            polygon['type'] = feature['geometry']
            polygon['bid'] = feature['id']
            polygon['roof_material'] = feature['properties']['roof_material']
            polygon['coordinates'] = feature['geometry']['coordinates']
            polygon['geojson'] = geojson
            polygons.append(polygon)
    except Exception as e:
        return e

    return polygons

In [ ]:
def transform_coordinates():
    pass

In [ ]:
def get_tif():
    pass

In [ ]:
def get_rooftop_array_after_mask():
    pass

In [ ]:
def convert_grayscale():
    pass

In [ ]:
def crop_center():
    pass

In [ ]:
def calculate_zonal_stats():
    pass

In [ ]:
def build_df():
    pass

In [18]:
# Obtain footprint dictionaries
# Returns a list of lists, where each list are polygons for one .tif file
pwex = pywren.default_executor()
futures = pwex.map(get_geojsons, GEOJSONS)
polygons = pywren.get_all_results(futures)